Env: Location_extraction

In [ ]:
import pandas as pd

# Setting up Nominatim 

In [ ]:
# Loading required packages for geocoding with Nominatim

import geopandas as gpd 
import geopy
import matplotlib.pyplot as plt
from functools import partial 
from geopy import distance
from geopy.distance import geodesic
from tqdm import tqdm, tqdm_notebook # progress bar

#initiate 
tqdm.pandas()

In [ ]:
# user_agent is used to overide restricts of using Nominatim default user_agent.
# If error *geopy has no attribute geocoders occurs, kill kernel and rerun line*

locator = geopy.geocoders.Nominatim(user_agent='mygeocoder')

In [ ]:
# Set to avoid the error of 'Too many requests 429 error'

from geopy.extra.rate_limiter import RateLimiter
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)

# return locations in english 

geocode = partial(locator.geocode, language = "en", timeout = 300000)

## Setting up folder loop

In [ ]:
#Defining files to be geocoded
unique_files = ['FAC.csv', 'GPE.csv','ORG.csv','LOC.csv', 'FAC_GPE.csv','LOC_GPE.csv', 'ORG_GPE.csv']

# Defining the address columns within corresponding files 

address_column_names = [elem.split('.')[0] for elem in unique_files]

# Geocoding with Nominatim

In [ ]:
ressources = 'Geocoding/unq_locations/'
results = 'Geocoding/nom_coords'
for unique_file,address_column_name  in zip(unique_files,address_column_names):
    ressource_path = ressources + unique_file
    result_path =  results +'/nom_geocoded_' + unique_file
    df = pd.read_csv(ressource_path)
    geocoded_values = []
    currentindex = 0
    for index, row in df.iloc[currentindex:].iterrows():
        geocoded_values.append(locator.geocode(row[address_column_name], language = 'en',  timeout = 30000,))
        #print("Added  " + row['clean_text'] )
        print("Index  " + str(currentindex) )
        currentindex = currentindex + 1
    df = df.assign(location=geocoded_values)
    df['nlong'] = df['location'].apply(lambda x: x.longitude if x else None)
    df['nlat'] = df['location'].apply(lambda x: x.latitude if x else None)
    df.to_csv(result_path)